In [4]:
import dask.dataframe as dd

import pandas as pd


In [2]:
import matplotlib.pyplot as plt

%matplotlib inline

In [8]:
from pathlib import Path
import tqdm

In [7]:
folders = []
for path in Path("/media/nightwing/ext/Data/raw_data/").glob("*"):
    folders.append(path)


**Number of files**

In [6]:
len(folders)


762

**Scores per parameters**

In [8]:
ad_tft = dd.read_csv("%s/main.csv" % folders[5])


In [36]:
ad_tft.columns

Index(['opponent', 'seed', 'num. of generations', 'bottleneck',
       'mutation probability', 'half size population', 'generation', 'index',
       'score', 'gene_0',
       ...
       'gene_195', 'gene_196', 'gene_197', 'gene_198', 'gene_199', 'gene_200',
       'gene_201', 'gene_202', 'gene_203', 'gene_204'],
      dtype='object', length=214)

In [9]:
grp = ad_tft.groupby(["bottleneck", "mutation probability", "half size population"])[
    "score"
].max()


In [10]:
grp.compute()


bottleneck  mutation probability  half size population
10          0.01                  10                      3.039024
            0.05                  10                      3.029268
            0.10                  10                      3.000000
20          0.01                  10                      3.000000
10          0.01                  15                      3.039024
20          0.05                  10                      3.000000
            0.10                  10                      3.000000
10          0.05                  15                      3.029268
            0.10                  15                      3.000000
20          0.01                  15                      3.039024
            0.05                  15                      3.024390
10          0.01                  20                      3.039024
20          0.10                  15                      3.000000
10          0.05                  20                      3.019512
       

In [12]:
tft = dd.read_csv("/media/nightwing/ext/Data/raw_data/Tit For Tat_nan/main.csv")


In [13]:
grp_tft = tft.groupby(["bottleneck", "mutation probability", "half size population"])[
    "score"
].max()


In [15]:
grp_tft.compute()


bottleneck  mutation probability  half size population
10          0.01                  10                      3.009756
            0.05                  10                      3.000000
            0.10                  10                      3.000000
20          0.01                  10                      3.000000
10          0.01                  15                      3.009756
20          0.05                  10                      3.000000
            0.10                  10                      3.000000
10          0.05                  15                      3.000000
            0.10                  15                      3.000000
20          0.01                  15                      3.009756
            0.05                  15                      3.000000
            0.10                  15                      3.000000
10          0.01                  20                      3.009756
            0.05                  20                      3.000000
       

In [16]:
coop = dd.read_csv("/media/nightwing/ext/Data/raw_data/Cooperator_nan/main.csv")


In [19]:
grp_coop = coop.groupby(["bottleneck", "mutation probability", "half size population"])[
    "score"
].max()


In [20]:
grp_coop.compute()


bottleneck  mutation probability  half size population
10          0.01                  10                      5.0
            0.05                  10                      5.0
            0.10                  10                      5.0
20          0.01                  10                      5.0
10          0.01                  15                      5.0
20          0.05                  10                      5.0
            0.10                  10                      5.0
10          0.05                  15                      5.0
            0.10                  15                      5.0
20          0.01                  15                      5.0
            0.05                  15                      5.0
            0.10                  15                      5.0
10          0.01                  20                      5.0
            0.05                  20                      5.0
            0.10                  20                      5.0
20          0.0

**Creating Training data**

In [5]:
last_generation = 2000
bottleneck, mutation_probability, half_size_population = 20, 0.01, 20
groubys = ["gene_{}".format(i) for i in range(205)]

In [9]:
for path in tqdm.tqdm(folders[:1]):
    strategy = str(path).split('/')[-1]
    
    df = dd.read_csv("%s/main.csv" % path)
    
    df = df[
        (df["generation"] == last_generation)
        & (df["bottleneck"] == bottleneck)
        & (df["mutation probability"] == mutation_probability)
        & (df["half size population"] == half_size_population)
    ]

    df = df[df["score"] == df["score"].max()][groubys].drop_duplicates().reset_index()
    df = df.compute(num_workers=4)
    df.to_csv('raw_data/%s_top_individuals.csv' % strategy)

100%|██████████| 1/1 [00:29<00:00, 29.07s/it]
